In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 2

@author: yaning
"""

import importlib
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math

# my own class files
import with_learning.learning_NN.Receptors as Receptors
import with_learning.learning_NN.Network as Network


importlib.reload(Receptors)
importlib.reload(Network)

path = "/home/yaning/Documents/Spiking_NN/threshold/"
np.set_printoptions(threshold=np.inf)

In [ ]:
with open(path + "XOR_dataset_black_white.pkl", "rb") as f:
    dataset, binary = pickle.load(f)

In [ ]:
dataset.shape

(8, 3, 6000)

In [ ]:
pointCount = 6000
deltaTms = 0.05
times = np.arange(pointCount) * deltaTms
initial_Vm = 1.3458754117369027

In [ ]:
# instead of using control i write those two functions :(have to copy it everytime i run)
all_synapses = []


def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def create_synapse(send_neuron, receive_neuron, type):
    
    # create receptors accordingly
    if type == "AMPA":
        # temporal solution for weight randomise
        # Receptors.LigandGatedChannelFactory.set_params()
        ampa_receptor = Receptors.AMPA(0.072, 1, -70, 1.35, 0.9, 1, 1, 1, 12, 10, 20, 10, 35, 7, 0.7, "AMPA")
        synapse = Network.Synapse(0.05, 0, send_neuron, receive_neuron, ampa_receptor)
        
    elif type == "AMPA+NMDA":
        # Receptors.LigandGatedChannelFactory.set_params()
        ampa_receptor = Receptors.AMPA(0.072, 1, -70, 1.35, 0.9, 1, 1, 1, 12, 10, 20, 10, 35, 7, 0.7, "AMPA")
        nmda_receptor = Receptors.NMDA(0.0012, 1, -70, 1.35, 0.9, 1, 1, 1, 13, 10, 20, 10, 15, 7, 0.7, "NMDA")
        synapse = Network.Synapse(0.05, 0, send_neuron, receive_neuron, ampa_receptor, nmda_receptor)
    
    elif type == "GABA":
        # Receptors.LigandGatedChannelFactory.set_params()
        # print(Receptors.LigandGatedChannelFactory.w_init_GABA)
        gaba_receptor = Receptors.GABA(0.004, 1, -140, 1.35, 0.9, 1, 1, 1, 12, 10, 20, 10, 20, 7, 0.7, "GABA")
        synapse = Network.Synapse(0.05, 0, send_neuron, receive_neuron, gaba_receptor)

    send_neuron.outgoing_synapses.append(synapse)
    receive_neuron.incoming_synapses.append(synapse)

    all_synapses.append(synapse)

def update_synapse_initial_values(infer_params):
    for synapse in all_synapses:
        for receptor in synapse.receptors:
            receptor.Vm = initial_Vm
            receptor.gP = 1
            
            receptor.e = infer_params["e"]
            receptor.u_se = infer_params["u_se"]
            receptor.g_decay = infer_params["g_decay"]
            receptor.g_rise = infer_params["g_rise"]
            receptor.w = infer_params["w"]
            receptor.tau_rec = infer_params["tau_rec"]
            receptor.tau_pre = infer_params["tau_pre"]
            receptor.tau_post = infer_params["tau_post"]

            if receptor.label == "GABA":
                receptor.gMax = infer_params["gMax_GABA"]
                receptor.tau_decay = infer_params["tau_decay_GABA"]
                receptor.tau_rise = infer_params["tau_rise_GABA"]
            
            elif receptor.label == "NMDA":
                receptor.tau_decay = infer_params["tau_decay_NMDA"]
                receptor.tau_rise = infer_params["tau_rise_NMDA"]
            
            elif receptor.label == "AMPA":
                receptor.tau_decay = infer_params["tau_decay_AMPA"]
                receptor.tau_rise = infer_params["tau_rise_AMPA"]

In [ ]:
# with open(path + "infer_params_names.pkl", "rb") as f:
#     infer_params, infer_names = pickle.load(f)

In [ ]:
# with learning every group has more neurons 
neuron_input_0 = Network.Neuron(deltaTms, 0, initial_Vm, "input_0")
neuron_input_1 = Network.Neuron(deltaTms, 0, initial_Vm, "input_1")
neuron_input_2 = Network.Neuron(deltaTms, 0, initial_Vm, "input_2")
# neuron_input_3 = Network.Neuron(deltaTms, 0, initial_Vm, "input_3")
# neuron_input_4 = Network.Neuron(deltaTms, 0, initial_Vm, "input_4")

neuron_excite_main_0 = Network.Neuron(deltaTms, 0, initial_Vm, "excite_main_0")
neuron_excite_main_1 = Network.Neuron(deltaTms, 0, initial_Vm, "excite_main_1")
neuron_excite_sub_0 = Network.Neuron(deltaTms, 0, initial_Vm, "excite_sub_0")
neuron_excite_sub_1 = Network.Neuron(deltaTms, 0, initial_Vm, "excite_sub_1")

neuron_inhibit_main_0 = Network.Neuron(deltaTms, 0, initial_Vm, "inhibit_main_0")
neuron_inhibit_main_1 = Network.Neuron(deltaTms, 0, initial_Vm, "inhibit_main_1")
neuron_inhibit_sub_0 = Network.Neuron(deltaTms, 0, initial_Vm, "inhibit_sub_0")
neuron_inhibit_sub_1 = Network.Neuron(deltaTms, 0, initial_Vm, "inhibit_sub_1")

neuron_output_0 = Network.Neuron(deltaTms, 0, initial_Vm, "output_0")
neuron_output_1 = Network.Neuron(deltaTms, 0, initial_Vm, "output_1")
# neuron_output_2 = Network.Neuron(deltaTms, 0, initial_Vm, "output_2")

neurons = [neuron_input_0, neuron_input_1, neuron_input_2, #neuron_input_3, neuron_input_4,
            neuron_excite_main_0, neuron_excite_main_1, 
            neuron_excite_sub_0, neuron_excite_sub_1, 
            neuron_inhibit_main_0, neuron_inhibit_main_1, 
            neuron_inhibit_sub_0, neuron_inhibit_sub_1,
            neuron_output_0, neuron_output_1#, neuron_output_2
            ]

neuron_names = ["input_0", "input_1", "input_2", #"input_3", "input_4",
                "excite_main_0", "excite_main_1", 
                "excite_sub_0", "excite_sub_1",
                "inhibit_main_0", "inhibit_main_1",
                "inhibit_sub_0", "inhibit_sub_1",
                "output_0", "output_1"#, "output_2"
                ]



#*********************full layer***************************
# ----------------first input layer------------------------
create_synapse(neuron_input_0, neuron_excite_main_0, "AMPA")
create_synapse(neuron_input_1, neuron_excite_main_0, "AMPA")
create_synapse(neuron_input_2, neuron_excite_main_0, "AMPA")
# create_synapse(neuron_input_3, neuron_excite_main_0, "AMPA")
# create_synapse(neuron_input_4, neuron_excite_main_0, "AMPA")

create_synapse(neuron_input_0, neuron_excite_main_1, "AMPA")
create_synapse(neuron_input_1, neuron_excite_main_1, "AMPA")
create_synapse(neuron_input_2, neuron_excite_main_1, "AMPA")
# create_synapse(neuron_input_3, neuron_excite_main_1, "AMPA")
# create_synapse(neuron_input_4, neuron_excite_main_1, "AMPA")

create_synapse(neuron_input_0, neuron_inhibit_main_0, "GABA")
create_synapse(neuron_input_1, neuron_inhibit_main_0, "GABA")
create_synapse(neuron_input_2, neuron_inhibit_main_0, "GABA")
# create_synapse(neuron_input_3, neuron_inhibit_main_0, "GABA")
# create_synapse(neuron_input_4, neuron_inhibit_main_0, "GABA")

create_synapse(neuron_input_0, neuron_inhibit_main_1, "GABA")
create_synapse(neuron_input_1, neuron_inhibit_main_1, "GABA")
create_synapse(neuron_input_2, neuron_inhibit_main_1, "GABA")
# create_synapse(neuron_input_3, neuron_inhibit_main_1, "GABA")
# create_synapse(neuron_input_4, neuron_inhibit_main_1, "GABA")

# ----------------self recurrent layer----------------
create_synapse(neuron_excite_main_0, neuron_excite_sub_0, "AMPA+NMDA")
create_synapse(neuron_excite_main_0, neuron_excite_sub_1, "AMPA+NMDA")

create_synapse(neuron_excite_main_1, neuron_excite_sub_0, "AMPA+NMDA")
create_synapse(neuron_excite_main_1, neuron_excite_sub_1, "AMPA+NMDA")

create_synapse(neuron_excite_sub_0, neuron_excite_main_0, "AMPA+NMDA")
create_synapse(neuron_excite_sub_0, neuron_excite_main_1, "AMPA+NMDA")

create_synapse(neuron_excite_sub_1, neuron_excite_main_0, "AMPA+NMDA")
create_synapse(neuron_excite_sub_1, neuron_excite_main_1, "AMPA+NMDA")
    

create_synapse(neuron_inhibit_main_0, neuron_inhibit_sub_0, "GABA")
create_synapse(neuron_inhibit_main_0, neuron_inhibit_sub_1, "GABA")

create_synapse(neuron_inhibit_main_1, neuron_inhibit_sub_0, "GABA")
create_synapse(neuron_inhibit_main_1, neuron_inhibit_sub_1, "GABA")

create_synapse(neuron_inhibit_sub_0, neuron_inhibit_main_0, "GABA")
create_synapse(neuron_inhibit_sub_0, neuron_inhibit_main_1, "GABA")

create_synapse(neuron_inhibit_sub_1, neuron_inhibit_main_0, "GABA")
create_synapse(neuron_inhibit_sub_1, neuron_inhibit_main_1, "GABA")

# --------------between excitatory and inhibitory----------------
create_synapse(neuron_excite_main_0, neuron_inhibit_main_0, "AMPA+NMDA")
create_synapse(neuron_excite_main_0, neuron_inhibit_main_1, "AMPA+NMDA")

create_synapse(neuron_excite_main_1, neuron_inhibit_main_0, "AMPA+NMDA")
create_synapse(neuron_excite_main_1, neuron_inhibit_main_1, "AMPA+NMDA")

create_synapse(neuron_inhibit_main_0, neuron_excite_main_0, "GABA")
create_synapse(neuron_inhibit_main_0, neuron_excite_main_1, "GABA")

create_synapse(neuron_inhibit_main_1, neuron_excite_main_0, "GABA")
create_synapse(neuron_inhibit_main_1, neuron_excite_main_1, "GABA")

# ----------------output layer----------------------
create_synapse(neuron_excite_main_0, neuron_output_0, "AMPA")
create_synapse(neuron_excite_main_1, neuron_output_0, "AMPA")

create_synapse(neuron_excite_main_0, neuron_output_1, "AMPA")
create_synapse(neuron_excite_main_1, neuron_output_1, "AMPA")

# create_synapse(neuron_excite_main_0, neuron_output_2, "AMPA")
# create_synapse(neuron_excite_main_1, neuron_output_2, "AMPA")

In [ ]:
binary

array([[0, 0, 0, 0],
       [0, 0, 1, 1],
       [0, 1, 0, 1],
       [0, 1, 1, 0],
       [1, 0, 0, 1],
       [1, 0, 1, 0],
       [1, 1, 0, 0],
       [1, 1, 1, 1]])

In [ ]:
dataset.shape[0]

8

In [ ]:
# run

for i in range(dataset.shape[0]-7):
    data = dataset[i+3]
    for neuron in neurons:
        neuron.erase(initial_Vm)

    print("ha")
    for t in range(pointCount):
        if data[0,t]:
            # print("this line runs")
            neuron_input_0.sending_signal()
            neuron_input_0.fire_tstep.append(t)
            
        if data[1,t]:
            neuron_input_1.sending_signal()
            neuron_input_1.fire_tstep.append(t)
            
                    
        if data[2,t]:
            neuron_input_2.sending_signal()
            neuron_input_2.fire_tstep.append(t)
                    
        # if data[3,t]:
        #     neuron_input_3.sending_signal()
        #     neuron_input_3.fire_tstep.append(t)

        # if data[4,t]:
        #     neuron_input_4.sending_signal()
        #     neuron_input_4.fire_tstep.append(t)



        # update the synapse states then each neuron
        # exclude checking the input neurons
        for neuron in neurons[3:]:
            if neuron.fire_tstep == []:
                last_fire = -2
            else:
                last_fire = neuron.fire_tstep[-1]
            fire = neuron.check_firing(t)
            if fire:
                if neuron.fire_tstep == [] or last_fire + 1 != t:
                    # print("this line runs")
                    neuron.update_weights(t)

            
            neuron.update()

    
        
        fired_synapse = []
        # set the synapse states back to 0
        for synapse in all_synapses:
            if synapse.state == 1:
                fired_synapse.append(synapse)
            synapse.state = 0
        # print(len(fired_synapse))

        desire = binary[i+3, -1]

        difference = len(neuron_output_0.fire_tstep) - len(neuron_output_1.fire_tstep)
        if  difference > 0:
            real = 0
        else: real = 1
        # rescaled_difference = sigmoid(difference)
        if desire == real:
            for synapse in fired_synapse:
                for receptor in synapse.receptors:
                    receptor.w = receptor.w * 1.001
        else:
            for synapse in fired_synapse:
                for receptor in synapse.receptors:
                    receptor.w = receptor.w * 0.999
        

        # Vm_neuron_output.append(neuron_excite_main_0.Vm - 70)
        # neuron_output_currents.append(neuron_output.I)
        # neuron_excite_main_0_currents.append(neuron_excite_main_0.I)
        # neuron_excite_main_firing_tstep.append(neuron_excite_main.fire_tstep)

ha


/home/yaning/Documents/Spiking_NN/with_learning/learning_NN/Receptors.py:79: RuntimeWarning: overflow encountered in exp
  self.m.alpha = .1*((25-self.Vm) / (np.exp((25-self.Vm)/10)-1))
/home/yaning/Documents/Spiking_NN/with_learning/learning_NN/Receptors.py:80: RuntimeWarning: overflow encountered in exp
  self.m.beta = 4*np.exp(-self.Vm/18)
/home/yaning/Documents/Spiking_NN/with_learning/learning_NN/Receptors.py:81: RuntimeWarning: overflow encountered in exp
  self.n.alpha = .01 * ((10-self.Vm) / (np.exp((10-self.Vm)/10)-1))
/home/yaning/Documents/Spiking_NN/with_learning/learning_NN/Receptors.py:83: RuntimeWarning: overflow encountered in exp
  self.h.alpha = .07*np.exp(-self.Vm/20)
/home/yaning/Documents/Spiking_NN/with_learning/learning_NN/Receptors.py:84: RuntimeWarning: overflow encountered in exp
  self.h.beta = 1/(np.exp((30-self.Vm)/10)+1)
/home/yaning/Documents/Spiking_NN/with_learning/learning_NN/Receptors.py:115: RuntimeWarning: overflow encountered in power
  self.gP = n

In [ ]:
for neuron in neurons:
    print(neuron.fire_tstep)

[]
[500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 968, 969, 970, 971, 972, 973, 974, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1202, 1203, 1204, 1205, 1206, 1207, 1208, 1209, 1210, 1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1436, 1437, 1438, 1439, 1440, 1441, 14

In [ ]:
# convert the firing array to a True and False array for the animation
firing_array = []
for i in neurons:
    converted = [t in i.fire_tstep for t in range(pointCount)]
    firing_array.append(converted)

firing_array = np.array(firing_array)


np.save(path + 'firing.npy', firing_array)

In [ ]:
for synapse in all_synapses[:]:
    for receptor in synapse.receptors:
        print(receptor.w)

12
12
12
12
12
12
12
12
12
12
12
12
12
13
12
13
12
13
12
13
12
13
12
13
12
13
12
13
12
12
12
12
12
12
12
12
12
13
12
13
12
13
12
13
12
12
12
12
12
12
12
12


In [ ]:
dataset.shape

(8, 5, 6000)

In [ ]:
num_correct = 0


for i in range(dataset.shape[0]):
    print(i)
    test = dataset[i]

    for neuron in neurons:
        neuron.erase(initial_Vm)

    for t in range(pointCount):
        if test[0,t]:
            # print("this line runs")
            neuron_input_0.sending_signal()
            neuron_input_0.fire_tstep.append(t)
            # print(neuron_input_0.fire_tstep)
            
        # if test[1,t]:
        #     neuron_input_1.sending_signal()
        #     neuron_input_1.fire_tstep.append(t)
            
        # if test[2,t]:
        #     neuron_input_2.sending_signal()
        #     neuron_input_2.fire_tstep.append(t)
            
        # if data[3,t]:
        #     neuron_input_3.sending_signal()
        #     neuron_input_3.fire_tstep.append(t)

        # if data[4,t]:
        #     neuron_input_4.sending_signal()
        #     neuron_input_4.fire_tstep.append(t)


        # update the synapse states then each neuron
        # exclude checking the input neurons and output neurons
        for neuron in neurons[1:]:
            if neuron.fire_tstep == []:
                last_fire = -2
            else:
                last_fire = neuron.fire_tstep[-1]
            fire = neuron.check_firing(t)
            # if fire:
                # if neuron.fire_tstep == [] or last_fire + 1 != t:
                    # print("this line runs")
                    # neuron.update_weights(t)



            
            neuron.update()

       # set the synapse states back to 0
        for synapse in all_synapses:
            synapse.state = 0

    
    # if sum(output_synapse_states_0) - sum(output_synapse_states_1) > 0:
    if len(neuron_output_0.fire_tstep) - len(neuron_output_1.fire_tstep) > 0:
        predict = 0
    else:
        predict = 1
    if predict == binary[i][-1]:
        print("yes")
        num_correct += 1

0
1
yes
2
yes
3
4
5
6
7
yes


In [ ]:
num_correct/32

0.09375

In [ ]:
len(neuron_output_1.fire_tstep)

1154

In [ ]:
def count_all_continuous_sequences(arr):
    if not arr:
        return 0

    # Sort the array to ensure it's in order
    arr = sorted(arr)

    count = 1  # Start with one sequence since there's at least one group
    new_arr = []
    new_arr.append(arr[0])
    # Iterate through the array
    for i in range(1, len(arr)):
        if arr[i] != arr[i - 1] + 1:
            # A break in continuity means a new sequence
            count += 1
            new_arr.append(arr[i])

    return count, new_arr

# Test the function
array = [1, 2, 3, 5,6, 8, 9, 15, 16, 17,18, 300]
count, new_arr = count_all_continuous_sequences(array)

In [ ]:
new_arr

[1, 5, 8, 15, 300]